In [23]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import re

from pathlib import Path

In [24]:
raw_data_dir = Path("/Users/me/tmp/data.payless.health/hospital_price_transparency_data/raw")
processed_data_dir = Path("/Users/me/tmp/data.payless.health/hospital_price_transparency_data/processed")
price_urls_dir = Path("/Users/me/tmp/data.payless.health/hospital_price_transparency_data/price_urls")

df_urls = pd.read_csv(price_urls_dir / "220106-dolthub.com-onefact-paylesshealth-hospitals.csv")
df_urls = df_urls.set_index("ccn")

In [4]:
df_urls

,name,state,city,homepage,cdm_indirect_url,cdm_url
ccn,,,,,,
154063,"NEUROPSYCHIATRIC HOSPITAL OF INDIANAPOLIS, LLC",IN,INDIANAPOLIS,https://www.neuropsychiatrichospitals.net/,https://www.neuropsychiatrichospitals.net/pric...,https://www.neuropsychiatrichospitals.net/wp-c...
154064,ASSURANCE HEALTH PSYCHIATRIC HOSPITAL,IN,INDIANAPOLIS,http://assurancehealthsystem.com/,NaN,NaN
154065,NEURO BEHAVIORAL HOSPITAL,IN,CROWN POINT,https://www.neuropsychiatrichospitals.net/,https://www.neuropsychiatrichospitals.net/pric...,https://www.neuropsychiatrichospitals.net/wp-c...
154066,BRIGHTWELL BEHAVIORAL HEALTH,IN,CLARKSVILLE,https://brightwellbehavioral.com/,NaN,NaN
154067,HENDRICKS BEHAVIORAL HOSPITAL,IN,PLAINFIELD,https://www.hendricksbehavioralhospital.com/,https://www.hendricksbehavioralhospital.com/,https://s3.amazonaws.com/ycubaa-production-mar...
...,...,...,...,...,...,...
491305,SHENANDOAH MEMORIAL HOSPITAL,VA,WOODSTOCK,https://www.valleyhealthlink.com/,https://www.valleyhealthlink.com/patients-visi...,https://www.valleyhealthlink.com/documents/pri...
491307,"PAGE MEMORIAL HOSPITAL, INC",VA,LURAY,https://www.valleyhealthlink.com/,https://www.valleyhealthlink.com/patients-visi...,https://www.valleyhealthlink.com/documents/pri...
452019,KINDRED HOSPITAL FORT WORTH- MANSFIELD,TX,MANSFIELD,https://www.kindredhospitals.com/,https://www.kindredhospitals.com/locations/lta...,https://www.kindredhospitals.com/docs/default-...


In [13]:
ccn_list = []
charge_list = []
description_list = []
for csv_file in list(raw_data_dir.glob("*.csv"))[:]:
    try:
        cms_certification_number = csv_file.name.split("_")[0]
        regex = re.compile(r"charge", flags=re.IGNORECASE)
        df = pd.read_csv(csv_file)
        df_charges = df.filter(regex=regex)
        dtypes_charges = df_charges.dtypes
        columns_charges = df_charges.columns
        charge_column = next(
            name
            for name, dtype in zip(columns_charges, dtypes_charges)
            if "desc" not in name.lower() and dtype == np.float64
        )
        regex = re.compile(r"desc|name", flags=re.IGNORECASE)
        df_description = df.filter(regex=regex)
        dtypes_description = df_description.dtypes
        columns_description = df_description.columns
        if len(columns_description) > 0:
            charges = df[charge_column].values.tolist()
            ccn_list.extend([cms_certification_number] * len(charges))
            charge_list.extend(charges)
            description_list.extend([str(x) for x in df[columns_description[0]].values])
    except:
        pass


/var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_65071/2034754970.py:8: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
/var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_65071/2034754970.py:8: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
/var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_65071/2034754970.py:8: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
/var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_65071/2034754970.py:8: DtypeWarning: Columns (3,8,9,10,11,12,13,17,18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)
/var/folders/2c/hmpw9qgd7fd3x0ms6548_f880000gn/T/ipykernel_65071/2034754970.py:8: DtypeWarning: Columns (3,4,5,11,13) have mixe

In [20]:
df = pd.DataFrame(zip(ccn_list, charge_list, description_list), columns=["ccn", "charge", "description"])

In [32]:
df["state"] = df["ccn"].map(df_urls["state"])

In [45]:
df

,ccn,charge,description,state
0,170120,796.60,FINE NEEDLE ASPIRATION,KS
1,170120,540.35,FINE NEEDLE ASPIR W US; 1ST LESION,KS
2,170120,540.35,FINE NEEDLE ASPIR W US; ADD LESION,KS
3,170120,540.35,FINE NEEDLE ASPIR W FLUORO; 1ST LESION,KS
4,170120,540.35,FINE NEEDLE ASPIR W FLUORO; ADD LESION,KS
...,...,...,...,...
562005,041310,989.00,METHYLENE BLUE 50 MG,AR
562006,041310,989.00,VISUALIZATION ADJUNCT,AR
562007,041310,620.60,"INJECTION, PENTAMIDINE ISETH",AR
562008,041310,620.60,PENTAM 300 MG VIAL,AR


In [47]:
table = pa.Table.from_pandas(df)
pq.write_to_dataset(table, root_path=processed_data_dir, partition_cols=["state"])  


In [50]:
# example of queriying the data in duckdb
import duckdb
import pyarrow.dataset as ds

# DuckDB setup 
con = duckdb.connect()
# enable automatic query parallelization
con.execute("PRAGMA threads=2")
# enable caching of parquet metadata
con.execute("PRAGMA enable_object_cache")

# link to parquet files using an Arrow Dataset
dataset = ds.dataset(str(processed_data_dir), partitioning="hive")

con.execute("SELECT * FROM 'dataset' LIMIT 5").df()

,ccn,charge,description,state
0,010040,134.66,LEUKOCYTE TRANSFUS,AL
1,010040,4.30,OXAZEPAM 15MG CAP,AL
2,010040,69.00,PIPER/TAZOB 1G/.125G,AL
3,010040,2353.36,LIMITED ED VISIT,AL
4,010040,192.50,GENTAMICIN UPTO 80MG,AL


In [54]:
con.execute("SELECT state, AVG(charge), STDDEV(charge) FROM 'dataset' GROUP BY state").df()

,state,avg(charge),stddev(charge)
0,AL,16597.971470,42731.521183
1,IA,678.395230,22336.844558
2,KS,3225.742224,11405.564281
3,MD,1020.543436,2873.712417
4,AR,11934.860829,12683.661548
